Look for where specific characters appear in the dataset:

In [1]:
# # ------- HAL --------
# output_folder = '/home/jnaiman/data/morgan/' # HAL, data
# model_save_dir = '/home/jnaiman/data/morgan/models/' # HAL, model

# ------- LOCAL --------
output_folder = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/' # HAL, data
model_save_dir = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/models/' # HAL, model


ender = '_small_words' # small has 100,000 for training, 5000 for dev


# its not 100% clear if we need this... setting a flag, but looks like we DO need it for memory issues
use_train_dev_size = True
train_size = 1000000
dev_size = 10000

In [16]:
import pickle
import torch
import numpy as np

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
char2i = pickle.load(open(output_folder + "data/char2i_new_pages"+ender+".pkl", "rb"))
i2char = pickle.load(open(output_folder + "data/i2char_new_pages"+ender+".pkl", "rb"))

In [22]:
train_source = torch.load(output_folder + "data/train_source_new_pages"+ender+".pt")[:train_size].to(device)#add to custom data
train_target = torch.load(output_folder + "data/train_target_new_pages"+ender+".pt")[:train_size].to(device)#add to custom data

In [42]:
#char_pick = '⋠'
char_pick = '⊟'
char_int = -1

for c,i in char2i.items():
    if char_pick == c:
        char_int = i
        
char2i[char_pick], char_int

(448, 448)

In [43]:
in_train = np.where(train_source == char_int)

In [46]:
# take the first one (or another):
isent = 0

senti = train_source[in_train[0][isent]]
sent_source = ''
for i in range(len(senti)):
    sent_source += i2char[int(senti[i].numpy())]
    
senti = train_target[in_train[0][isent]]
sent_target = ''
for i in range(len(senti)):
    sent_target += i2char[int(senti[i].numpy())]

print("source:")
print(sent_source)
print("target:")
print(sent_target)

source:
<START> ⋀↑↴↾⊜∣⋪⋃⋯⋪⊓∣⋪⋋↾∆⋦⊽∪⊔−∣∣∊∁⋅≏∐∏∣⊃⋅≏∏∶↔↾∏⋅⊟⋯ <END><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
target:
<START> After our first campaign, Borucki et al. ) )<END><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>


From seq2seq output:

In [47]:
#Model: Seq2Seq Transformer
Source_index =  {'\t': 3, '\n': 4, ' ': 5, '!': 6, '"': 7, '#': 8, '$': 9, '%': 10, '&': 11, "'": 12, '(': 13, ')': 14, '*': 15, '+': 16, ',': 17, '-': 18, '.': 19, '/': 20, '0': 21, '1': 22, '2': 23, '3': 24, '4': 25, '5': 26, '6': 27, '7': 28, '8': 29, '9': 30, ':': 31, ';': 32, '<': 33, '<UNK>': 34, '=': 35, '>': 36, '?': 37, '@': 38, 'A': 39, 'B': 40, 'C': 41, 'D': 42, 'E': 43, 'F': 44, 'G': 45, 'H': 46, 'I': 47, 'J': 48, 'K': 49, 'L': 50, 'M': 51, 'N': 52, 'O': 53, 'P': 54, 'Q': 55, 'R': 56, 'S': 57, 'T': 58, 'U': 59, 'V': 60, 'W': 61, 'X': 62, 'Y': 63, 'Z': 64, '[': 65, '\\': 66, ']': 67, '^': 68, '_': 69, '`': 70, 'a': 71, 'b': 72, 'c': 73, 'd': 74, 'e': 75, 'f': 76, 'g': 77, 'h': 78, 'i': 79, 'j': 80, 'k': 81, 'l': 82, 'm': 83, 'n': 84, 'o': 85, 'p': 86, 'q': 87, 'r': 88, 's': 89, 't': 90, 'u': 91, 'v': 92, 'w': 93, 'x': 94, 'y': 95, 'z': 96, '{': 97, '|': 98, '}': 99, '~': 100, '\xa0': 101, '¡': 102, '¢': 103, '£': 104, '¥': 105, '§': 106, '©': 107, 'ª': 108, '«': 109, '\xad': 110, '®': 111, '¯': 112, '°': 113, '±': 114, '³': 115, '´': 116, 'µ': 117, '¶': 118, '»': 119, '¼': 120, '½': 121, '¿': 122, 'À': 123, 'Á': 124, 'Â': 125, 'Ã': 126, 'Å': 127, 'É': 128, 'Ê': 129, 'Í': 130, 'Ð': 131, 'Ò': 132, 'Ó': 133, 'Õ': 134, 'Ö': 135, 'Ü': 136, 'Þ': 137, 'à': 138, 'á': 139, 'â': 140, 'ä': 141, 'ç': 142, 'è': 143, 'é': 144, 'ê': 145, 'ë': 146, 'í': 147, 'î': 148, 'ï': 149, 'ò': 150, 'ó': 151, 'ô': 152, 'ö': 153, 'ø': 154, 'ú': 155, 'ü': 156, 'ý': 157, 'ć': 158, 'Č': 159, 'č': 160, 'İ': 161, 'ł': 162, 'ń': 163, 'ő': 164, 'ř': 165, 'ş': 166, 'š': 167, 'ź': 168, 'Ż': 169, 'ż': 170, 'ž': 171, '̆': 172, '̇': 173, '΄': 174, 'Ά': 175, 'Έ': 176, 'Ί': 177, 'Ό': 178, 'Ώ': 179, 'Α': 180, 'Β': 181, 'Γ': 182, 'Δ': 183, 'Ε': 184, 'Ζ': 185, 'Η': 186, 'Θ': 187, 'Ι': 188, 'Κ': 189, 'Λ': 190, 'Μ': 191, 'Ν': 192, 'Ξ': 193, 'Ο': 194, 'Π': 195, 'Ρ': 196, 'Σ': 197, 'Τ': 198, 'Υ': 199, 'Φ': 200, 'Χ': 201, 'Ω': 202, 'ά': 203, 'έ': 204, 'ή': 205, 'ί': 206, 'α': 207, 'β': 208, 'γ': 209, 'δ': 210, 'ε': 211, 'ζ': 212, 'η': 213, 'θ': 214, 'ι': 215, 'κ': 216, 'λ': 217, 'μ': 218, 'ν': 219, 'ξ': 220, 'ο': 221, 'π': 222, 'ρ': 223, 'ς': 224, 'σ': 225, 'τ': 226, 'υ': 227, 'φ': 228, 'χ': 229, 'ψ': 230, 'ω': 231, 'ϊ': 232, 'ό': 233, 'ύ': 234, 'ώ': 235, 'ϐ': 236, 'ϱ': 237, 'ἀ': 238, 'ἁ': 239, 'ἂ': 240, 'ἃ': 241, 'ἄ': 242, 'ἆ': 243, 'Ἀ': 244, 'Ἅ': 245, 'ἐ': 246, 'ἓ': 247, 'Ἐ': 248, 'ἣ': 249, 'Ἠ': 250, 'Ἡ': 251, 'ἰ': 252, 'ἱ': 253, 'ἲ': 254, 'ἳ': 255, 'ἴ': 256, 'ἵ': 257, 'ἶ': 258, 'ἷ': 259, 'Ἰ': 260, 'Ἱ': 261, 'Ἴ': 262, 'ὁ': 263, 'ὃ': 264, 'ὅ': 265, 'Ὀ': 266, 'Ὁ': 267, 'Ὃ': 268, 'Ὅ': 269, 'ὐ': 270, 'ὑ': 271, 'ὓ': 272, 'ὖ': 273, 'ὠ': 274, 'Ὦ': 275, 'ὰ': 276, 'ὲ': 277, 'ὴ': 278, 'ὶ': 279, 'ὸ': 280, 'ᾱ': 281, 'ᾳ': 282, '᾿': 283, 'ῃ': 284, 'ῇ': 285, 'ῖ': 286, 'ῥ': 287, 'ῦ': 288, 'Ῥ': 289, '–': 290, '—': 291, '‘': 292, '’': 293, '“': 294, '”': 295, '€': 296, '™': 297, '↑': 298, '→': 299, '↓': 300, '↔': 301, '↕': 302, '↖': 303, '↗': 304, '↘': 305, '↙': 306, '↛': 307, '↜': 308, '↝': 309, '↠': 310, '↡': 311, '↢': 312, '↣': 313, '↤': 314, '↥': 315, '↦': 316, '↧': 317, '↨': 318, '↩': 319, '↪': 320, '↫': 321, '↭': 322, '↰': 323, '↱': 324, '↲': 325, '↳': 326, '↴': 327, '↵': 328, '↶': 329, '↷': 330, '↸': 331, '↺': 332, '↻': 333, '↼': 334, '↽': 335, '↾': 336, '↿': 337, '⇀': 338, '⇁': 339, '⇂': 340, '⇃': 341, '⇄': 342, '⇆': 343, '⇇': 344, '⇈': 345, '⇉': 346, '⇍': 347, '⇖': 348, '⇜': 349, '⇠': 350, '⇡': 351, '⇥': 352, '⇪': 353, '∀': 354, '∁': 355, '∂': 356, '∃': 357, '∆': 358, '∇': 359, '∉': 360, '∊': 361, '∋': 362, '∍': 363, '∎': 364, '∏': 365, '∐': 366, '∑': 367, '−': 368, '∓': 369, '∔': 370, '∕': 371, '∖': 372, '∙': 373, '∞': 374, '∟': 375, '∠': 376, '∡': 377, '∢': 378, '∣': 379, '∤': 380, '∥': 381, '∩': 382, '∪': 383, '∫': 384, '∱': 385, '∶': 386, '∷': 387, '∸': 388, '∺': 389, '∼': 390, '∾': 391, '∿': 392, '≀': 393, '≃': 394, '≈': 395, '≊': 396, '≋': 397, '≏': 398, '≒': 399, '≓': 400, '≖': 401, '≙': 402, '≚': 403, '≜': 404, '≝': 405, '≞': 406, '≟': 407, '≡': 408, '≣': 409, '≤': 410, '≥': 411, '≦': 412, '≧': 413, '≨': 414, '≩': 415, '≪': 416, '≯': 417, '≱': 418, '≲': 419, '≳': 420, '≴': 421, '≵': 422, '≶': 423, '≸': 424, '≹': 425, '≺': 426, '≻': 427, '≼': 428, '≽': 429, '≿': 430, '⊀': 431, '⊂': 432, '⊃': 433, '⊆': 434, '⊇': 435, '⊊': 436, '⊋': 437, '⊍': 438, '⊏': 439, '⊐': 440, '⊑': 441, '⊒': 442, '⊓': 443, '⊔': 444, '⊖': 445, '⊙': 446, '⊜': 447, '⊟': 448, '⊡': 449, '⊢': 450, '⊣': 451, '⊤': 452, '⊥': 453, '⊨': 454, '⊪': 455, '⊰': 456, '⊱': 457, '⊲': 458, '⊳': 459, '⊴': 460, '⊸': 461, '⊹': 462, '⊺': 463, '⊻': 464, '⊼': 465, '⊽': 466, '⊾': 467, '⋀': 468, '⋁': 469, '⋂': 470, '⋃': 471, '⋅': 472, '⋈': 473, '⋉': 474, '⋊': 475, '⋋': 476, '⋍': 477, '⋎': 478, '⋏': 479, '⋔': 480, '⋖': 481, '⋗': 482, '⋚': 483, '⋜': 484, '⋝': 485, '⋞': 486, '⋟': 487, '⋠': 488, '⋡': 489, '⋣': 490, '⋤': 491, '⋥': 492, '⋦': 493, '⋨': 494, '⋪': 495, '⋮': 496, '⋯': 497, '⋰': 498, '⋱': 499, '<PAD>': 0, '<START>': 1, '<END>': 2}
Target_index = {3: '\t', 4: '\n', 5: ' ', 6: '!', 7: '"', 8: '#', 9: '$', 10: '%', 11: '&', 12: "'", 13: '(', 14: ')', 15: '*', 16: '+', 17: ',', 18: '-', 19: '.', 20: '/', 21: '0', 22: '1', 23: '2', 24: '3', 25: '4', 26: '5', 27: '6', 28: '7', 29: '8', 30: '9', 31: ':', 32: ';', 33: '<', 34: '<UNK>', 35: '=', 36: '>', 37: '?', 38: '@', 39: 'A', 40: 'B', 41: 'C', 42: 'D', 43: 'E', 44: 'F', 45: 'G', 46: 'H', 47: 'I', 48: 'J', 49: 'K', 50: 'L', 51: 'M', 52: 'N', 53: 'O', 54: 'P', 55: 'Q', 56: 'R', 57: 'S', 58: 'T', 59: 'U', 60: 'V', 61: 'W', 62: 'X', 63: 'Y', 64: 'Z', 65: '[', 66: '\\', 67: ']', 68: '^', 69: '_', 70: '`', 71: 'a', 72: 'b', 73: 'c', 74: 'd', 75: 'e', 76: 'f', 77: 'g', 78: 'h', 79: 'i', 80: 'j', 81: 'k', 82: 'l', 83: 'm', 84: 'n', 85: 'o', 86: 'p', 87: 'q', 88: 'r', 89: 's', 90: 't', 91: 'u', 92: 'v', 93: 'w', 94: 'x', 95: 'y', 96: 'z', 97: '{', 98: '|', 99: '}', 100: '~', 101: '\xa0', 102: '¡', 103: '¢', 104: '£', 105: '¥', 106: '§', 107: '©', 108: 'ª', 109: '«', 110: '\xad', 111: '®', 112: '¯', 113: '°', 114: '±', 115: '³', 116: '´', 117: 'µ', 118: '¶', 119: '»', 120: '¼', 121: '½', 122: '¿', 123: 'À', 124: 'Á', 125: 'Â', 126: 'Ã', 127: 'Å', 128: 'É', 129: 'Ê', 130: 'Í', 131: 'Ð', 132: 'Ò', 133: 'Ó', 134: 'Õ', 135: 'Ö', 136: 'Ü', 137: 'Þ', 138: 'à', 139: 'á', 140: 'â', 141: 'ä', 142: 'ç', 143: 'è', 144: 'é', 145: 'ê', 146: 'ë', 147: 'í', 148: 'î', 149: 'ï', 150: 'ò', 151: 'ó', 152: 'ô', 153: 'ö', 154: 'ø', 155: 'ú', 156: 'ü', 157: 'ý', 158: 'ć', 159: 'Č', 160: 'č', 161: 'İ', 162: 'ł', 163: 'ń', 164: 'ő', 165: 'ř', 166: 'ş', 167: 'š', 168: 'ź', 169: 'Ż', 170: 'ż', 171: 'ž', 172: '̆', 173: '̇', 174: '΄', 175: 'Ά', 176: 'Έ', 177: 'Ί', 178: 'Ό', 179: 'Ώ', 180: 'Α', 181: 'Β', 182: 'Γ', 183: 'Δ', 184: 'Ε', 185: 'Ζ', 186: 'Η', 187: 'Θ', 188: 'Ι', 189: 'Κ', 190: 'Λ', 191: 'Μ', 192: 'Ν', 193: 'Ξ', 194: 'Ο', 195: 'Π', 196: 'Ρ', 197: 'Σ', 198: 'Τ', 199: 'Υ', 200: 'Φ', 201: 'Χ', 202: 'Ω', 203: 'ά', 204: 'έ', 205: 'ή', 206: 'ί', 207: 'α', 208: 'β', 209: 'γ', 210: 'δ', 211: 'ε', 212: 'ζ', 213: 'η', 214: 'θ', 215: 'ι', 216: 'κ', 217: 'λ', 218: 'μ', 219: 'ν', 220: 'ξ', 221: 'ο', 222: 'π', 223: 'ρ', 224: 'ς', 225: 'σ', 226: 'τ', 227: 'υ', 228: 'φ', 229: 'χ', 230: 'ψ', 231: 'ω', 232: 'ϊ', 233: 'ό', 234: 'ύ', 235: 'ώ', 236: 'ϐ', 237: 'ϱ', 238: 'ἀ', 239: 'ἁ', 240: 'ἂ', 241: 'ἃ', 242: 'ἄ', 243: 'ἆ', 244: 'Ἀ', 245: 'Ἅ', 246: 'ἐ', 247: 'ἓ', 248: 'Ἐ', 249: 'ἣ', 250: 'Ἠ', 251: 'Ἡ', 252: 'ἰ', 253: 'ἱ', 254: 'ἲ', 255: 'ἳ', 256: 'ἴ', 257: 'ἵ', 258: 'ἶ', 259: 'ἷ', 260: 'Ἰ', 261: 'Ἱ', 262: 'Ἴ', 263: 'ὁ', 264: 'ὃ', 265: 'ὅ', 266: 'Ὀ', 267: 'Ὁ', 268: 'Ὃ', 269: 'Ὅ', 270: 'ὐ', 271: 'ὑ', 272: 'ὓ', 273: 'ὖ', 274: 'ὠ', 275: 'Ὦ', 276: 'ὰ', 277: 'ὲ', 278: 'ὴ', 279: 'ὶ', 280: 'ὸ', 281: 'ᾱ', 282: 'ᾳ', 283: '᾿', 284: 'ῃ', 285: 'ῇ', 286: 'ῖ', 287: 'ῥ', 288: 'ῦ', 289: 'Ῥ', 290: '–', 291: '—', 292: '‘', 293: '’', 294: '“', 295: '”', 296: '€', 297: '™', 298: '↑', 299: '→', 300: '↓', 301: '↔', 302: '↕', 303: '↖', 304: '↗', 305: '↘', 306: '↙', 307: '↛', 308: '↜', 309: '↝', 310: '↠', 311: '↡', 312: '↢', 313: '↣', 314: '↤', 315: '↥', 316: '↦', 317: '↧', 318: '↨', 319: '↩', 320: '↪', 321: '↫', 322: '↭', 323: '↰', 324: '↱', 325: '↲', 326: '↳', 327: '↴', 328: '↵', 329: '↶', 330: '↷', 331: '↸', 332: '↺', 333: '↻', 334: '↼', 335: '↽', 336: '↾', 337: '↿', 338: '⇀', 339: '⇁', 340: '⇂', 341: '⇃', 342: '⇄', 343: '⇆', 344: '⇇', 345: '⇈', 346: '⇉', 347: '⇍', 348: '⇖', 349: '⇜', 350: '⇠', 351: '⇡', 352: '⇥', 353: '⇪', 354: '∀', 355: '∁', 356: '∂', 357: '∃', 358: '∆', 359: '∇', 360: '∉', 361: '∊', 362: '∋', 363: '∍', 364: '∎', 365: '∏', 366: '∐', 367: '∑', 368: '−', 369: '∓', 370: '∔', 371: '∕', 372: '∖', 373: '∙', 374: '∞', 375: '∟', 376: '∠', 377: '∡', 378: '∢', 379: '∣', 380: '∤', 381: '∥', 382: '∩', 383: '∪', 384: '∫', 385: '∱', 386: '∶', 387: '∷', 388: '∸', 389: '∺', 390: '∼', 391: '∾', 392: '∿', 393: '≀', 394: '≃', 395: '≈', 396: '≊', 397: '≋', 398: '≏', 399: '≒', 400: '≓', 401: '≖', 402: '≙', 403: '≚', 404: '≜', 405: '≝', 406: '≞', 407: '≟', 408: '≡', 409: '≣', 410: '≤', 411: '≥', 412: '≦', 413: '≧', 414: '≨', 415: '≩', 416: '≪', 417: '≯', 418: '≱', 419: '≲', 420: '≳', 421: '≴', 422: '≵', 423: '≶', 424: '≸', 425: '≹', 426: '≺', 427: '≻', 428: '≼', 429: '≽', 430: '≿', 431: '⊀', 432: '⊂', 433: '⊃', 434: '⊆', 435: '⊇', 436: '⊊', 437: '⊋', 438: '⊍', 439: '⊏', 440: '⊐', 441: '⊑', 442: '⊒', 443: '⊓', 444: '⊔', 445: '⊖', 446: '⊙', 447: '⊜', 448: '⊟', 449: '⊡', 450: '⊢', 451: '⊣', 452: '⊤', 453: '⊥', 454: '⊨', 455: '⊪', 456: '⊰', 457: '⊱', 458: '⊲', 459: '⊳', 460: '⊴', 461: '⊸', 462: '⊹', 463: '⊺', 464: '⊻', 465: '⊼', 466: '⊽', 467: '⊾', 468: '⋀', 469: '⋁', 470: '⋂', 471: '⋃', 472: '⋅', 473: '⋈', 474: '⋉', 475: '⋊', 476: '⋋', 477: '⋍', 478: '⋎', 479: '⋏', 480: '⋔', 481: '⋖', 482: '⋗', 483: '⋚', 484: '⋜', 485: '⋝', 486: '⋞', 487: '⋟', 488: '⋠', 489: '⋡', 490: '⋣', 491: '⋤', 492: '⋥', 493: '⋦', 494: '⋨', 495: '⋪', 496: '⋮', 497: '⋯', 498: '⋰', 499: '⋱', 0: '<PAD>', 1: '<START>', 2: '<END>'}

In [74]:
# only_in_source = []
# only_in_source_vals = []
# for s,i in Source_index.items():
#     if s not in list(Target_index.values()):
#         only_in_source.append(s)
#         only_in_source_vals.append(i)
# only_in_source_vals = np.unique(only_in_source_vals)
only_in_source_vals = np.arange(100,len(Source_index)) # things that start to get into "weird" characters

In [76]:
#only_in_source_vals

Count how many times one of these "mystery" characters appears:

In [77]:
isent_source = 0
for i,t in enumerate(train_source):
    inds = np.intersect1d(only_in_source_vals, t)
    if len(inds)>0:
        isent_source += 1

In [78]:
isent_source, len(train_source) # number of sentence windows that have odd characters, length of full thing

(78662, 1000000)

In [79]:
isent_source/len(train_source)*100 # percent

7.866199999999999

In [64]:
# take the first one (or another):
isent = 1000000//2

senti = train_source[isent]
sent_source = ''
for i in range(len(senti)):
    sent_source += i2char[int(senti[i].numpy())]
    
senti = train_target[isent]
sent_target = ''
for i in range(len(senti)):
    sent_target += i2char[int(senti[i].numpy())]

print("source:")
print(sent_source)
print("target:")
print(sent_target)

source:
<START>ually probable for this star.<END><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
target:
<START>ually probable for this star.<END><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
